In [1]:
from pkg.utils.utils_python3 import *
import random

CONVERTED_PATH = "./data/converted"
SCENE_FILENAME = "scene.pkl"
JOINT_NUM = 13
gtimer = GlobalTimer()
# ## Load Global params
DATASET_LIST = sorted(os.listdir(CONVERTED_PATH))
trainset_num = None
if trainset_num is not None:
    TRAINSET_LIST =  DATASET_LIST[:trainset_num]
    TESTSET_LIST = DATASET_LIST[trainset_num:]
else:
    TRAINSET_LIST = ['20201214-165211', '20201216-021416', '20201218-024611']
    TESTSET_LIST = ['20201208-121454', '20201212-232318', '20201213-061207']
    
# trainset
SCENE_TUPLE_LIST = []
for DATASET in TRAINSET_LIST:
    CURRENT_PATH = os.path.join(CONVERTED_PATH, DATASET)
    #Iterate world
    WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))
    gtimer.reset()
    for WORLD in WORLD_LIST:
        WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
        # Iterate scene
        SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))
        for SCENE in SCENE_LIST:
            SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
            ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))
            for ACTION in ACTION_LIST:
                N_action = get_action_count(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION)
                for i_act in range(N_action):
                    SCENE_TUPLE_LIST.append((CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM))
                    #gtimer.tic("load_scene_data")
                    #scene_data, success, skey = load_scene_data(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM)
                    #gtimer.toc("load_scene_data")
train_set = SCENE_TUPLE_LIST
N_train = len(train_set)

# trainset
SCENE_TUPLE_LIST = []
for DATASET in TESTSET_LIST:
    CURRENT_PATH = os.path.join(CONVERTED_PATH, DATASET)
    #Iterate world
    WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))
    gtimer.reset()
    for WORLD in WORLD_LIST:
        WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
        # Iterate scene
        SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))
        for SCENE in SCENE_LIST:
            SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
            ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))
            for ACTION in ACTION_LIST:
                N_action = get_action_count(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION)
                for i_act in range(N_action):
                    SCENE_TUPLE_LIST.append((CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM))
                    #gtimer.tic("load_scene_data")
                    #scene_data, success, skey = load_scene_data(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM)
                    #gtimer.toc("load_scene_data")
test_set = SCENE_TUPLE_LIST
N_test = len(test_set)
print(f'Train/Test: {N_train}, {N_test}')

Train/Test: 69484, 6276


In [2]:
joint_num = 13
N_vtx_box = 3*8
N_mask_box = 1
N_joint_box = joint_num
N_label_box = N_vtx_box+N_mask_box+N_joint_box
N_vtx_cyl = 3*2+1
N_mask_cyl = 1
N_joint_cyl = joint_num
N_label_cyl = N_vtx_cyl+N_mask_cyl+N_joint_cyl
N_vtx_init = 3*8
N_mask_init = 1
N_joint_init = joint_num
N_label_init = N_vtx_init+N_mask_init+N_joint_init
N_vtx_goal = 3*8
N_mask_goal = 1
N_joint_goal = joint_num
N_label_goal = N_vtx_goal+N_mask_goal+N_joint_goal
N_joint_label = 6*joint_num
N_cell_label = N_label_box+N_label_cyl+N_label_init+N_label_goal + N_joint_label
N_BEGIN_CYL = N_vtx_box+N_mask_box+N_joint_box
N_BEGIN_INIT = N_BEGIN_CYL+N_vtx_cyl+N_mask_cyl+N_joint_cyl
N_BEGIN_GOAL = N_BEGIN_INIT+N_vtx_init+N_mask_init+N_joint_init
def separate_dat(scene_data):
    cbox = scene_data[:,:,:,:, :N_vtx_box]
    cbox_m = scene_data[:,:,:,:, N_vtx_box]
    cbox_j = np.repeat(scene_data[:,:,:,:, N_vtx_box+1:N_vtx_box+1+N_joint_box], 6, axis=-1)
    ccyl = scene_data[:,:,:,:, N_BEGIN_CYL:N_BEGIN_CYL+N_vtx_cyl]
    ccyl_m = scene_data[:,:,:,:, N_BEGIN_CYL+N_vtx_cyl]
    ccyl_j = np.repeat(scene_data[:,:,:,:, N_BEGIN_CYL+N_vtx_cyl+1:N_BEGIN_CYL+N_vtx_cyl+1+N_joint_cyl], 6, axis=-1)
    ibox = scene_data[:,:,:,:, N_BEGIN_INIT:N_BEGIN_INIT+N_vtx_box]
    ibox_m = scene_data[:,:,:,:, N_BEGIN_INIT+N_vtx_box]
    ibox_j = np.repeat(scene_data[:,:,:,:, N_BEGIN_INIT+N_vtx_box+1:N_BEGIN_INIT+N_vtx_box+1+N_joint_init], 6, axis=-1)
    gbox = scene_data[:,:,:,:, N_BEGIN_GOAL:N_BEGIN_GOAL+N_vtx_box]
    gbox_m = scene_data[:,:,:,:, N_BEGIN_GOAL+N_vtx_box]
    gbox_j = np.repeat(scene_data[:,:,:,:, N_BEGIN_GOAL+N_vtx_box+1:N_BEGIN_GOAL+N_vtx_box+1+N_joint_goal], 6, axis=-1)
    joints = scene_data[:,:,:,:,-N_joint_label:]
    return (cbox, np.expand_dims(cbox_m, axis=-1), cbox_j, 
            ccyl, np.expand_dims(ccyl_m, axis=-1), ccyl_j,
            ibox, np.expand_dims(ibox_m, axis=-1), ibox_j,
            gbox, np.expand_dims(gbox_m, axis=-1), gbox_j, 
            joints)

In [3]:
import tensorflow as tf

from tensorflow.keras import layers as KL
from tensorflow.keras import Model, Sequential
import datetime

class DenseBN(KL.Layer):
    def __init__(self, units, name, use_bias=True, activation="relu"):
        super(DenseBN, self).__init__()
        self.dense = KL.Dense(units, name='dense_' + name, use_bias=use_bias)
        self.bn = KL.BatchNormalization(name='bn_' + name)
        self.ac = KL.Activation(activation)
        
    def call(self, x, training=False):
        x = self.dense(x)
        x = self.bn(x, training=training)
        x = self.ac(x)
        return x
        

class IdentityBlock(KL.Layer):
    def __init__(self, kernel_size, filters, stage, block,
                 use_bias=True, ConvLayer=KL.Conv2D):
        super(IdentityBlock, self).__init__()
        nb_filter1, nb_filter2, nb_filter3 = filters
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        
        self.conv1 = ConvLayer(nb_filter1, 1, name=conv_name_base + '2a',
                               use_bias=use_bias)
        self.bn1 = KL.BatchNormalization(name=bn_name_base + '2a')
        self.ac1 = KL.Activation('relu')
        
        self.conv2 = ConvLayer(nb_filter2, kernel_size, padding="same",
                               name=conv_name_base + '2b', use_bias=use_bias)
        self.bn2 = KL.BatchNormalization(name=bn_name_base + '2b')
        self.ac2 = KL.Activation('relu')
        
        self.conv3 = ConvLayer(nb_filter3, 1, name=conv_name_base + '2c',
                               use_bias=use_bias)
        self.bn3 = KL.BatchNormalization(name=bn_name_base + '2c')
        self.add = KL.Add()
        self.ac3 = KL.Activation('relu', name='res' + str(stage) + block + '_out')
        
    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.ac1(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.ac2(x)

        x = self.conv3(x)
        x = self.bn3(x, training=training)

        x = self.add([x, input_tensor])
        x = self.ac3(x)
        return x
    
class IdentityBlock3D(IdentityBlock):
    def __init__(self, *args, **kwargs):
        super(IdentityBlock3D, self).__init__(*args, ConvLayer=KL.Conv3D, **kwargs)
    
    

class ConvBlock(KL.Layer):
    def __init__(self, kernel_size, filters, stage, block, strides=2,
                 use_bias=True, ConvLayer=KL.Conv2D):
        super(ConvBlock, self).__init__()
        nb_filter1, nb_filter2, nb_filter3 = filters
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        
        self.conv1 = ConvLayer(nb_filter1, 1, strides=strides, 
                               name=conv_name_base + '2a', use_bias=use_bias)
        self.bn1 = KL.BatchNormalization(name=bn_name_base + '2a')
        self.ac1 = KL.Activation('relu')
        
        self.conv2 = ConvLayer(nb_filter2, kernel_size, padding="same",
                               name=conv_name_base + '2b', use_bias=use_bias)
        self.bn2 = KL.BatchNormalization(name=bn_name_base + '2b')
        self.ac2 = KL.Activation('relu')
        
        self.conv3 = ConvLayer(nb_filter3, 1, name=conv_name_base + '2c',
                               use_bias=use_bias)
        self.bn3 = KL.BatchNormalization(name=bn_name_base + '2c')

        self.convs = ConvLayer(nb_filter3, 1, strides=strides,
                               name=conv_name_base + '1', use_bias=use_bias)
        self.bns = KL.BatchNormalization(name=bn_name_base + '1')
    
        self.add = KL.Add()
        self.ac3 = KL.Activation('relu', name='res' + str(stage) + block + '_out')
        
    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.ac1(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.ac2(x)

        x = self.conv3(x)
        x = self.bn3(x, training=training)
        
        shortcut = self.convs(input_tensor)
        shortcut = self.bns(shortcut, training=training)

        x = self.add([x, shortcut])
        x = self.ac3(x)
        return x
    
class ConvBlock3D(ConvBlock):
    def __init__(self, *args, **kwargs):
        super(ConvBlock3D, self).__init__(*args, ConvLayer=KL.Conv3D, **kwargs)

class ResNet(KL.Layer):
    def __init__(self, architecture="resnet50", stage0=None, stage1=None, stage2=[64, 64, 256], stage3=[128, 128, 512], 
                 stage4=[256, 256, 1024], stage5=None, 
                 ConvLayer=KL.Conv2D, ZeroPadding=KL.ZeroPadding2D, MaxPool=KL.MaxPooling2D, 
                 ConvBlock=ConvBlock, IdentityBlock=IdentityBlock, input_size=(15,15,15), joint_num=13, batch_size=16,
                 Nwdh=(15,15,15)):
        super(ResNet, self).__init__()
        """Build a ResNet graph.
            architecture: Can be resnet50 or resnet101
            stage5: recommanded default values = [512, 512, 2048]. If None, stage5 of the network is not created
        """
        assert architecture in ["resnet50", "resnet101"]
        self.stage1 = stage1 is not None
        self.stage5 = stage5 is not None
        self.stage0 = stage0 is not None
        self.input_size = input_size
        self.joint_num = joint_num
        self.batch_size = batch_size
        if self.stage0:
            self.pre_jnta = ConvBlock(3, stage0, stage=0, block='ja', strides=1)
            self.pre_jntb = IdentityBlock(3, stage0, stage=0, block='jb')
            
            self.pre_boxa = ConvBlock(3, stage0, stage=0, block='ba', strides=1)
            self.pre_boxb = IdentityBlock(3, stage0, stage=0, block='bb')
            self.pre_cyla = ConvBlock(3, stage0, stage=0, block='ca', strides=1)
            self.pre_cylb = IdentityBlock(3, stage0, stage=0, block='cb')
            self.pre_obsa = ConvBlock(3, stage0, stage=0, block='oa', strides=1)
            self.pre_obsb = IdentityBlock(3, stage0, stage=0, block='ob')
            
            self.pre_inia = ConvBlock(3, stage0, stage=0, block='ia', strides=1)
            self.pre_inib = IdentityBlock(3, stage0, stage=0, block='ib')
            self.pre_gola = ConvBlock(3, stage0, stage=0, block='ga', strides=1)
            self.pre_golb = IdentityBlock(3, stage0, stage=0, block='gb')
            
            self.concat = KL.Concatenate()
        
        if self.stage1:
            # Stage 1  output size = 1/1
            self.cb1a = ConvBlock(3, stage1, stage=1, block='a', strides=1)
            self.cb1b = IdentityBlock(3, stage1, stage=1, block='b')
            stride_2 = 2
        else:
            # Stage 1 original output size = 1/4
            self.cv1 = ConvLayer(64, 7, strides=2, 
                                 name='conv1', padding="same", use_bias=True)
            self.bn1 = KL.BatchNormalization(name='bn_conv1')
            self.ac1 = KL.Activation('relu')
            self.mp1 = MaxPool(3, strides=2, padding="same")
            stride_2 = 1
        
        # Stage 2  output size = 1/stride_2
        self.cb2a = ConvBlock(3, stage2, stage=2, block='a', strides=stride_2)
        self.cb2b = IdentityBlock(3, stage2, stage=2, block='b')
        
        # Stage 3  output size = 1/2
        self.cb3a = ConvBlock(3, stage3, stage=3, block='a')
        self.cb3b = IdentityBlock(3, stage3, stage=3, block='b')
        
        # Stage 4  output size = 1/2
        self.cb4a = ConvBlock(3, stage4, stage=4, block='a')
        self.cb4b = IdentityBlock(3, stage4, stage=4, block='b')
            
        # Stage 5  output size = 1/2
        if self.stage5:
            self.cb5a = ConvBlock(3, stage5, stage=5, block='a')
            self.cb5b = IdentityBlock(3, stage5, stage=5, block='b')

    def call(self, input_image, training=False):
        if self.stage0:
            cbox, cbox_m, cbox_j, ccyl, ccyl_m, ccyl_j, \
                ibox, ibox_m, ibox_j, gbox, gbox_m, gbox_j, \
                joints = input_image
            
            cbox = self.pre_boxa(cbox, training=training)
            cbox = self.pre_boxb(cbox, training=training)*cbox_m
            cbox_jnt = self.pre_jnta(joints*cbox_j, training=training)
            cbox_jnt = self.pre_jntb(cbox_jnt, training=training)
            cbox = self.concat([cbox, cbox_jnt])
            
            ccyl = self.pre_cyla(ccyl, training=training)
            ccyl = self.pre_cylb(ccyl, training=training)*ccyl_m
            ccyl_jnt = self.pre_jnta(joints*ccyl_j, training=training)
            ccyl_jnt = self.pre_jntb(ccyl_jnt, training=training)
            ccyl = self.concat([ccyl, ccyl_jnt])
            
            obs = self.concat([cbox, ccyl])
            obs = self.pre_obsa(obs, training=training)
            obs = self.pre_obsb(obs, training=training)
            
            ibox = self.pre_boxa(ibox, training=training)
            ibox = self.pre_boxb(ibox, training=training)*ibox_m
            ibox_jnt = self.pre_jnta(joints*ibox_j, training=training)
            ibox_jnt = self.pre_jntb(ibox_jnt, training=training)
            ibox = self.concat([ibox, ibox_jnt])
            
            gbox = self.pre_boxa(gbox, training=training)
            gbox = self.pre_boxb(gbox, training=training)*gbox_m
            gbox_jnt = self.pre_jnta(joints*gbox_j, training=training)
            gbox_jnt = self.pre_jntb(gbox_jnt, training=training)
            gbox = self.concat([gbox, gbox_jnt])
            
            x = self.concat([obs, ibox, gbox])
        else:
            x = input_image
        # Stage 1
        if self.stage1:
            C1 = x = self.cb1a(x, training=training)
            C1 = x = self.cb1b(x, training=training)
        else:
            x = self.cv1(x)
            x = self.bn1(x, training=training)
            x = self.ac1(x)
            C1 = x = self.mp1(x)
        
        # Stage 2
        C2 = x = self.cb2a(x, training=training)
        C2 = x = self.cb2b(x, training=training)
        
        # Stage 3
        C3 = x = self.cb3a(x, training=training)
        C3 = x = self.cb3b(x, training=training)
        
        # Stage 4
        C4 = x = self.cb4a(x, training=training)
        C4 = x = self.cb4b(x, training=training)
            
        # Stage 5
        if self.stage5:
            C5 = x = self.cb5a(x, training=training)
            C5 = x = self.cb5b(x, training=training)
            return [C1, C2, C3, C4, C5]
        else:
            return [C1, C2, C3, C4]
            
    
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [4]:
class ResNetModelTP(Model):
    def __init__(self, 
                 ConvLayer=KL.Conv3D, ZeroPadding=KL.ZeroPadding3D, MaxPool=KL.GlobalMaxPool3D, 
                 ConvBlock=ConvBlock3D, IdentityBlock=IdentityBlock3D):
        super(ResNetModelTP, self).__init__()
        self.resnet = ResNet(architecture="resnet50", 
                             stage0 = [64,64,128], stage1=[256, 256, 512], 
                             stage2=[256, 256, 512], stage3=[256, 256, 512], 
                             stage4=[256, 256, 512], stage5=[256, 256, 512], 
                             ConvLayer=ConvLayer, ZeroPadding=ZeroPadding, MaxPool=MaxPool, 
                             ConvBlock=ConvBlock, IdentityBlock=IdentityBlock)
        self.gp1 = MaxPool()
        self.gp2 = MaxPool()
        self.gp3 = MaxPool()
        self.gp4 = MaxPool()
        self.flatten = KL.Flatten()
        self.dens11 = DenseBN(512, "dens11")
        self.dens21 = DenseBN(512, "dens21")
        self.dens31 = DenseBN(512, "dens31")
        self.dens41 = DenseBN(512, "dens41")
        self.dens51 = DenseBN(512, "dens51")
        self.dens12 = KL.Dense(256) # DenseBN(512, "dens1")
        self.dens22 = KL.Dense(256) # DenseBN(512, "dens2")
        self.dens32 = KL.Dense(256) # DenseBN(512, "dens3")
        self.dens42 = KL.Dense(256) # DenseBN(512, "dens4")
        self.dens52 = KL.Dense(256) # DenseBN(512, "dens5")
        self.add = KL.Add()
        self.dens_int1 = DenseBN(256, "dens_int1")
        self.dens_int2 = DenseBN(256, "dens_int2")
        self.dens_out = KL.Dense(2)

    def call(self, x, training=False):
        C1, C2, C3, C4, C5 = self.resnet(x, training=training)
        f1 = self.dens11(self.gp1(C1), training=training)
        f1 = self.dens12(f1, training=training)
        f2 = self.dens21(self.gp2(C2), training=training)
        f2 = self.dens22(f2, training=training)
        f3 = self.dens31(self.gp3(C3), training=training)
        f3 = self.dens32(f3, training=training)
        f4 = self.dens41(self.gp4(C4), training=training)
        f4 = self.dens42(f4, training=training)
        f5 = self.dens51(self.flatten(C5), training=training)
        f5 = self.dens52(f5, training=training)
        x = self.add([f1,f2,f3,f4,f5])
        # x = f5
        x = self.dens_int1(x, training=training)
        x = self.dens_int2(x, training=training)
        x = self.dens_out(x)
        return x

# Create an instance of the model
model = ResNetModelTP()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
model_log_dir = 'logs/gradient_tape/' + current_time + '/model.h5/'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)


In [6]:
EPOCHS_S = 0
EPOCHS_E = 100
BATCH_SIZE = 16
LOG_STEP = 100
first_run = True
for epoch in range(EPOCHS_S, EPOCHS_E):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    random.shuffle(train_set)
    i_step = 0
    scene_batch, success_batch = [], []
    for scene_tuple in train_set:
        i_step += 1
        scene_data, success, skey = load_scene_data(*scene_tuple)
        scene_batch.append(scene_data)
        success_batch.append(success)
        if i_step%BATCH_SIZE==0:
            data_batch_sep = separate_dat(np.array(scene_batch, dtype=np.float32))
            train_step(data_batch_sep, np.array(success_batch, dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("train step - {}/{}        ".format(i_step, N_train), end = '\r')
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    i_step = 0
    scene_batch, success_batch = [], []
    for scene_tuple in test_set:
        i_step += 1
        scene_data, success, skey = load_scene_data(*scene_tuple)
        scene_batch.append(scene_data)
        success_batch.append(success)
        if i_step%BATCH_SIZE==0:
            data_batch_sep = separate_dat(np.array(scene_batch, dtype=np.float32))
            if first_run:
                first_run = False
                tf.summary.trace_on(graph=True, profiler=True)
                test_step(data_batch_sep, np.array(success_batch,dtype=np.int))
                with test_summary_writer.as_default():
                    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=test_log_dir)
            else:
                test_step(data_batch_sep, np.array(success_batch,dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
            
    model.save(model_log_dir)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Epoch 1, Loss: 0.5669188499450684, Accuracy: 71.14376831054688, Test Loss: 0.9285745620727539, Test Accuracy: 64.93941497802734
Epoch 2, Loss: 0.5090783834457397, Accuracy: 74.88053131103516, Test Loss: 1.4349582195281982, Test Accuracy: 64.93941497802734
Epoch 3, Loss: 0.48742732405662537, Accuracy: 76.31707763671875, Test Loss: 0.8111686110496521, Test Accuracy: 64.93941497802734
Epoch 4, Loss: 0.47193098068237305, Accuracy: 77.38082122802734, Test Loss: 1.190899133682251, Test Accuracy: 64.97130584716797


KeyboardInterrupt: 

### train log
* dense bn for end stage: saturated output
* dense batch norm for each stage: 74% for train, saturated for test
* dense linear for each stage: 73% for train, saturated for test
* dense bn relu - dense linear for each stage: 74% for train, saturated for test
* dense bn relu - dense linear for each stage (size down): 74% for train, saturated for test
* separate input - dense bn relu - dense linear for each stage : 50epoch - 98.5% for train, max 74% for test
* separate input - dense bn relu - dense linear for each stage : 60k data - 95% for train, 75% for test (20201218-203453)
* 32 batch - OOM
* compact network, 32batch - >84/69, E false non-trainning layer (20201220-124752)
* compact network + 1idblocks, 16batch -> test satrated
* dropout - 
* dense^2 relu/sigmoid for each stage: 
* dense^2 relu/sigmoid x2 for each stage: 